In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/bbc_data.csv")
df.head()

,data,labels
0,Musicians to tackle US red tape Musicians gro...,entertainment
1,"U2s desire to be number one U2, who have won ...",entertainment
2,Rocker Doherty in on-stage fight Rock singer ...,entertainment
3,Snicket tops US box office chart The film ada...,entertainment
4,"Oceans Twelve raids box office Oceans Twelve,...",entertainment


In [ ]:
col= df.columns
col

Index(['data', 'labels'], dtype='object')

In [ ]:
cat = df['labels'].unique()
cat

array(['entertainment', 'business', 'sport', 'politics', 'tech'],
      dtype=object)

In [ ]:
df.shape

(2225, 2)

In [ ]:
val = df['labels'].value_counts()
val

labels
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64

In [ ]:
pip install transformers

In [ ]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

In [ ]:
texts = df['data'].tolist()
labels = df['labels'].tolist()

In [ ]:
label_to_int = {label: idx for idx, label in enumerate(set(labels_train))}
int_to_label = {idx: label for label, idx in label_to_int.items()}

In [ ]:
labels_train_int = [label_to_int[label] for label in labels_train]
labels_test_int = [label_to_int[label] for label in labels_test]

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(labels)))


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(texts_train, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(texts_test, truncation=True, padding=True, max_length=128)


In [ ]:
class BBCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = BBCDataset(train_encodings, labels_train_int)
test_dataset = BBCDataset(test_encodings, labels_test_int)


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


In [ ]:
trainer.train()

Step,Training Loss
10,1.500100
20,0.984700
30,0.631700
40,0.351000
50,0.226600
60,0.098500
70,0.147400
80,0.230200
90,0.103200
100,0.086000


Step,Training Loss
10,1.500100
20,0.984700
30,0.631700
40,0.351000
50,0.226600
60,0.098500
70,0.147400
80,0.230200
90,0.103200
100,0.086000


TrainOutput(global_step=378, training_loss=0.14649131025862758, metrics={'train_runtime': 8241.3369, 'train_samples_per_second': 0.729, 'train_steps_per_second': 0.046, 'total_flos': 395071890909696.0, 'train_loss': 0.14649131025862758, 'epoch': 3.0})

In [ ]:
trainer.evaluate()


{'eval_loss': 0.1645289808511734,
 'eval_runtime': 93.4852,
 'eval_samples_per_second': 2.385,
 'eval_steps_per_second': 0.15,
 'epoch': 3.0}

In [ ]:
input_text = "The technology conference was very insightful."

# Tokenize and encode the text
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=512)

In [ ]:
model.eval()

# Make predictions
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Convert logits to probabilities
probs = torch.nn.functional.softmax(logits, dim=-1)

# Get the predicted class
predicted_class = torch.argmax(probs, dim=-1).item()

In [ ]:
# Define the list of labels
labels = ['entertainment', 'business', 'sport', 'politics', 'tech']

# Get the label for the predicted class
predicted_label = labels[predicted_class]

print(f'Predicted class: {predicted_label}')

Predicted class: entertainment
